# Demo de predicción combinada

Este notebook prueba el **predictor combinado** (modelo tabular + modelo de texto)
usando la función `predecir_viabilidad` de `src.models.predictor`.

Asegúrate de haber entrenado y guardado los modelos:

- `models/modelo_tabular.joblib`
- `models/modelo_texto.joblib`

In [1]:
from pathlib import Path
import sys

# Suponiendo que este notebook está en la carpeta `notebooks/`
ROOT = Path().resolve().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.models.predictor import predecir_viabilidad, cargar_modelos

model_tab, model_text, feat_names = cargar_modelos()
print("✅ Modelos cargados.")
print("Umbral demo texto:", getattr(model_text, "_thr_demo", 0.5))
print("Tiene feature_names tabulares?:", feat_names is not None)

✅ Modelos cargados.
Umbral demo texto: 0.45
Tiene feature_names tabulares?: True


In [2]:
def probar_startup(descripcion, alpha=0.5, **campos_tabulares):
    """
    Helper para probar el predictor combinado.

    - descripcion: texto libre.
    - alpha: peso del modelo tabular en la combinación (0 a 1).
    - campos_tabulares: cualquier campo adicional que alimente el modelo tabular
      (monto_financiado, num_rondas, tamano_equipo, exp_fundadores, presencia_redes,
       inversores_destacados, tiempo_fundacion, sector, ubicacion, estado_operativo).
    """
    raw = {"descripcion": descripcion}
    raw.update(campos_tabulares)

    res = predecir_viabilidad(raw, alpha=alpha)

    print("\n==============================")
    print("Descripción:", descripcion)
    print(f"Prob_texto: {res['proba_texto']:.3f} (thr={res['thr_texto']:.2f}) ->", res["pred_texto"])
    if res["proba_tabular"] is not None:
        print(f"Prob_tabular: {res['proba_tabular']:.3f} ->", res["pred_tabular"])
    else:
        print("Prob_tabular: (no se usó, faltan campos)")
    print(f"Prob_combinada: {res['proba_combinada']:.3f} ->", res["pred_combinada"])
    print("Usa modelo tabular?:", res["usa_tabular"])
    print("Interpretación:")
    print(res["explicacion_texto"]["interpretacion"])
    print("Justificación técnica:")
    print(res["explicacion_texto"]["justificacion"])
    return res

## Ejemplos rápidos

Primero probamos con solo **descripción de texto** (sin llenar campos tabulares).

Luego puedes jugar pasando también campos tabulares, por ejemplo:

```python
probar_startup(
    desc_viable,
    alpha=0.6,
    monto_financiado="500000",
    num_rondas=2,
    tamano_equipo=12,
    exp_fundadores=8,
    presencia_redes=80,
    inversores_destacados="sí",
    tiempo_fundacion="2019",
    sector="Fintech",
    ubicacion="Lima",
    estado_operativo="activo",
)
```

In [3]:
# Ejemplos esperados VIABLE (solo texto)
desc1 = (
    "Healthtech de telemedicina con 300 médicos registrados, más de 10 000 "
    "consultas realizadas, contratos con 3 clínicas privadas, MRR de 40k USD "
    "y crecimiento mensual del 25%."
)

desc2 = (
    "Plataforma B2B de logística para pymes con 2500 empresas activas, tickets promedio "
    "de 300 USD, retención del 85%, MRR superior a 60k USD y acuerdos firmados con dos "
    "cadenas de supermercados."
)

desc3 = (
    "Marketplace de educación en línea con 15 000 usuarios registrados, 1200 suscriptores "
    "de pago, crecimiento de 35% mensual en ventas y ronda semilla cerrada de 500k USD "
    "liderada por un fondo regional."
)

# Ejemplos esperados NO VIABLE (solo texto)
desc4 = (
    "Idea de app para compartir gastos entre amigos, solo en fase conceptual, sin clientes, "
    "sin usuarios y sin ingresos. Aún no hay prototipo funcional ni validaciones con el mercado."
)

desc5 = (
    "Emprendimiento de delivery ecológico que tuvo algunos pedidos puntuales pero actualmente "
    "está en pausa, sin clientes activos, sin ingresos y con problemas financieros por deudas acumuladas."
)

# Pruebas (solo texto)
res1 = probar_startup(desc1, alpha=0.5)
res2 = probar_startup(desc2, alpha=0.5)
res3 = probar_startup(desc3, alpha=0.5)
res4 = probar_startup(desc4, alpha=0.5)
res5 = probar_startup(desc5, alpha=0.5)


Descripción: Healthtech de telemedicina con 300 médicos registrados, más de 10 000 consultas realizadas, contratos con 3 clínicas privadas, MRR de 40k USD y crecimiento mensual del 25%.
Prob_texto: 0.493 (thr=0.45) -> True
Prob_tabular: (no se usó, faltan campos)
Prob_combinada: 0.493 -> True
Usa modelo tabular?: False
Interpretación:
La startup **sí parece viable** (probabilidad 0.49) porque menciona señales positivas como mensual 25, MRR y ingresos recurrentes (MRR).
Justificación técnica:
Aportes: mensual 25:0.255; MRR:0.244; __tiene_mrr__:0.244; __crecimiento_medio__:0.078; crecimiento:0.033

Descripción: Plataforma B2B de logística para pymes con 2500 empresas activas, tickets promedio de 300 USD, retención del 85%, MRR superior a 60k USD y acuerdos firmados con dos cadenas de supermercados.
Prob_texto: 0.433 (thr=0.45) -> False
Prob_tabular: (no se usó, faltan campos)
Prob_combinada: 0.433 -> False
Usa modelo tabular?: False
Interpretación:
La startup **no parece viable** (proba

In [ ]:
# ============================================================
# Ejemplos usando también el MODELO TABULAR
# (se pasan campos estructurados además de la descripción)
# ============================================================

# 1) Startup claramente VIABLE (mucho dinero, equipo decente, activos)
res_tab_1 = probar_startup(
    descripcion=(
        "Plataforma SaaS de facturación electrónica para pymes con contratos "
        "recurrentes y buen crecimiento."
    ),
    alpha=0.5,
    monto_financiado=500000,        # USD aprox
    num_rondas=2,
    tamano_equipo=18,
    exp_fundadores=12,              # años de experiencia acumulada
    presencia_redes=85,             # buena presencia en redes
    inversores_destacados="si",
    tiempo_fundacion=2018,
    sector="E-commerce",
    ubicacion="Lima",
    estado_operativo="activo",
)

print("=== Caso TAB 1 – Viable fuerte (texto + tabular) ===")
print("Proba texto     :", round(res_tab_1["proba_texto"], 3), "| pred_texto     :", res_tab_1["pred_texto"])
print("Proba tabular   :", res_tab_1["proba_tabular"], "| pred_tabular   :", res_tab_1["pred_tabular"])
print("Proba combinada :", round(res_tab_1["proba_combinada"], 3), "| pred_combinada:", res_tab_1["pred_combinada"])
print("→ Interpretación:", res_tab_1["explicacion_texto"]["interpretacion"])
print()


# 2) Startup INTERMEDIA (números ok pero no tan fuertes)
res_tab_2 = probar_startup(
    descripcion=(
        "Aplicación móvil para reservas de canchas deportivas con algunos clientes frecuentes."
    ),
    alpha=0.5,
    monto_financiado=70000,
    num_rondas=1,
    tamano_equipo=7,
    exp_fundadores=5,
    presencia_redes=40,
    inversores_destacados="no",
    tiempo_fundacion=2021,
    sector="Logistics",
    ubicacion="Trujillo",
    estado_operativo="activo",
)

print("=== Caso TAB 2 – Viable intermedio (texto + tabular) ===")
print("Proba texto     :", round(res_tab_2["proba_texto"], 3), "| pred_texto     :", res_tab_2["pred_texto"])
print("Proba tabular   :", res_tab_2["proba_tabular"], "| pred_tabular   :", res_tab_2["pred_tabular"])
print("Proba combinada :", round(res_tab_2["proba_combinada"], 3), "| pred_combinada:", res_tab_2["pred_combinada"])
print("→ Interpretación:", res_tab_2["explicacion_texto"]["interpretacion"])
print()


# 3) Startup claramente NO VIABLE (poca plata, equipo chico, cerrada)
res_tab_3 = probar_startup(
    descripcion=(
        "Emprendimiento de marketplace local que estuvo activo un año pero ha sido cerrado."
    ),
    alpha=0.5,
    monto_financiado=8000,
    num_rondas=0,
    tamano_equipo=3,
    exp_fundadores=1,
    presencia_redes=5,
    inversores_destacados="no",
    tiempo_fundacion=2023,
    sector="E-commerce",
    ubicacion="Chiclayo",
    estado_operativo="cerrado",
)

print("=== Caso TAB 3 – No viable (texto + tabular) ===")
print("Proba texto     :", round(res_tab_3["proba_texto"], 3), "| pred_texto     :", res_tab_3["pred_texto"])
print("Proba tabular   :", res_tab_3["proba_tabular"], "| pred_tabular   :", res_tab_3["pred_tabular"])
print("Proba combinada :", round(res_tab_3["proba_combinada"], 3), "| pred_combinada:", res_tab_3["pred_combinada"])
print("→ Interpretación:", res_tab_3["explicacion_texto"]["interpretacion"])
print()


# 4) Ejemplo con CAMPOS TABULARES INCOMPLETOS (simula usuario que no sabe todo)
res_tab_4 = probar_startup(
    descripcion=(
        "Healthtech básica de teleconsultas en fase de crecimiento, con algunos contratos B2B."
    ),
    alpha=0.5,
    monto_financiado=None,          # el usuario no sabe el monto
    num_rondas=None,
    tamano_equipo=10,
    exp_fundadores=None,
    presencia_redes=60,
    inversores_destacados="no",
    tiempo_fundacion=2020,
    sector="AI/ML",
    ubicacion="Lima",
    estado_operativo="activo",
)

print("=== Caso TAB 4 – Parcial (faltan datos tabulares) ===")
print("Proba texto     :", round(res_tab_4["proba_texto"], 3), "| pred_texto     :", res_tab_4["pred_texto"])
print("Proba tabular   :", res_tab_4["proba_tabular"], "| pred_tabular   :", res_tab_4["pred_tabular"])
print("Proba combinada :", round(res_tab_4["proba_combinada"], 3), "| pred_combinada:", res_tab_4["pred_combinada"])
print("→ Interpretación:", res_tab_4["explicacion_texto"]["interpretacion"])


Categorías codificadas: ['sector', 'ubicacion', 'estado_operativo', 'antiguedad_bucket', 'sector_x_estado'] | Shape: (1, 32)

Descripción: Plataforma SaaS de facturación electrónica para pymes con contratos recurrentes y buen crecimiento.
Prob_texto: 0.140 (thr=0.45) -> False
Prob_tabular: 0.910 -> True
Prob_combinada: 0.525 -> True
Usa modelo tabular?: True
Interpretación:
La startup **no parece viable** (probabilidad 0.14) porque no aparecen suficientes señales de tracción o madurez; lo más relevante fue crecimiento sostenido.
Justificación técnica:
Aportes: crecimiento:0.089
=== Caso TAB 1 – Viable fuerte (texto + tabular) ===
Proba texto     : 0.14 | pred_texto     : False
Proba tabular   : 0.9099901532688923 | pred_tabular   : True
Proba combinada : 0.525 | pred_combinada: True
→ Interpretación: La startup **no parece viable** (probabilidad 0.14) porque no aparecen suficientes señales de tracción o madurez; lo más relevante fue crecimiento sostenido.

Categorías codificadas: ['sec

c:\Users\carlo\OneDrive\Escritorio\Proyecto Machine Learning\mcproyect\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\carlo\OneDrive\Escritorio\Proyecto Machine Learning\mcproyect\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\carlo\OneDrive\Escritorio\Proyecto Machine Learning\mcproyect\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
